In [74]:
from keras.models import Model
from keras.layers import Input,LSTM,Dense
import numpy as np


In [75]:
batch_size = 64
epochs = 1
latent_dim = 256
num_samples = 10000
data_path = 'cmn.txt'

In [76]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path,'r',encoding = 'utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples,len(lines) - 1)]:
#     print(line)
    input_text, target_text = line.split('\t')
#     print(input_text,target_text)
    target_text = '\t' + target_text+ '\n' #目标文本以tab开始，\n结束
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
    
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max(len(txt) for txt in input_texts)
max_decoder_seq_length = max(len(txt) for txt in target_texts)

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)



Number of samples: 10000
Number of unique input tokens: 73
Number of unique output tokens: 2626
Max sequence length for inputs: 31
Max sequence length for outputs: 22


In [77]:
#查找token
input_token_index = dict(
    [char, i] for i, char in enumerate(input_characters))
target_token_index = dict(
    [char, i] for i, char in enumerate(target_characters))

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype = 'float32') 
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype = 'float32')
decoder_target_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens), 
    dtype = 'float32')

for i, (input_text, target_text) in  enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.   
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
#         #解码的目标数据要比解码的输入数据快一步
        if t > 0:
            decoder_target_data[i, t-1, target_token_index[char]] = 1.
        


In [78]:
#定义输入数据并处理
encoder_inputs = Input(shape = (None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state = True)
encoder_output,state_h,state_c = encoder(encoder_inputs)
#去除encoder_output,只保留state
encoder_states = [state_h,state_c]

#设置解码，将编码的输出状态作为初始状态
decoder_inputs = Input(shape = (None, num_decoder_tokens))
#返回所有的输出序列以及所有的中间状态
decoder_lstm = LSTM(latent_dim, 
                    return_sequences = True,
                    return_state = True)
decoder_outputs, _, _  = decoder_lstm(decoder_inputs,
                                      initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, 
                      activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)



In [79]:
#建立模型，将输入数据的编码和解码作为输入张量，将解码的目标数据作为输出张量
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#编译模型
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy')
#训练模型
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size = batch_size,
          epochs = epochs,
          validation_split = 0.2)
#保存模型为hdf5文件
model.save('seq2seq.h5')


Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 14s 2ms/step - loss: 2.0337 - val_loss: 2.5031


/home/works/dl-tools/miniconda2/envs/python3.env/lib/python3.6/site-packages/keras/engine/topology.py:2344: UserWarning: Layer lstm_14 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_13/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_13/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [80]:
#推理模式（采样）
#1、将输入进行编码作为解码器的初始状态
#2、用这个初始状态和一个开始序列运行解码器的第一步并作为目标数据，将输出作为下一个目标token
#3、用当前的目标token和状态重复进行此过程

#定义采样模型
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_h = Input(shape = (latent_dim, ))
decoder_state_c = Input(shape = (latent_dim, ))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state =decoder_states_inputs )
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

#反转查找token到易读状态
reverse_input_char_index = dict(
    [i, char] for char, i in (input_token_index.items()))
reverse_target_char_index = dict(
    [i, char] for char, i in (target_token_index.items()))



In [83]:
#解码输出
def decode_sequence(input_seq):
    #将输入数据编码为状态向量
    states_value = encoder_model.predict(input_seq)
    
    #生成空的长度为1的目标序列
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #将目标序列的第一个字符填充为开始字符'\t'
    target_seq[0, 0, target_token_index['\t']] = 1

    #循环采样
    stop_condition = False
    decoded_sequence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        #对token进行采样
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sequence += sampled_char
        
        #退出循环条件：找到停止词'\n'或者达到最大长度
        if (sampled_char == '\n' or 
            len(decoded_sequence) > max_decoder_seq_length):
            stop_condition = True
        #更新目标序列
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1

        #更新状态
        
        states_value = [h, c]
    return decoded_sequence
     


In [84]:
#用一个序列尝试一下
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sequence = decode_sequence(input_seq)
    print('-')
    print('input sequence :', input_seq)
    print('Ddecoded sequence :', decoded_sequence)



-
input sequence : [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
Ddecoded sequence : 我姆是了。。

-
input sequence : [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
Ddecoded sequence : 我姆是了。。

-
input sequence : [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
Ddecoded sequence : 我姆是了。。

-
input sequence : [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
Ddecoded sequence : 我姆是了。。

-
input sequence : [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
 

input sequence : [[[0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
Ddecoded sequence : 我姆是了。。

-
input sequence : [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
Ddecoded sequence : 我姆是了。。

-
input sequence : [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
Ddecoded sequence : 我姆是了。。

-
input sequence : [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
Ddecoded sequence : 我姆是了。。

-
input sequence : [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [

-
input sequence : [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
Ddecoded sequence : 我姆是了。。

-
input sequence : [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
Ddecoded sequence : 我姆是了。。

-
input sequence : [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
Ddecoded sequence : 我姆是了。。

-
input sequence : [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
Ddecoded sequence : 我姆是了。。

-
input sequence : [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
 